In [2]:
from quantrion.asset.file import CSVUSStock

import asyncio
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

In [3]:
stock = CSVUSStock("AAPL", "files/apple_historical.csv")

In [4]:
start = stock.dt.now() - pd.Timedelta("1000d")
data = await stock.bars.get(start, freq="3min")

In [5]:
start_train, end_train = data.index[0], data.index[int(data.shape[0] * 0.8)]
start_test, end_test = end_train + pd.Timedelta("1min"), data.index[-1]
data_train = data[:end_train]
data_test = data[start_test:]

In [126]:
from enum import auto

start, end = pd.Timestamp("2022-08-17", tz=stock.tz), pd.Timestamp(
    "2022-08-18", tz=stock.tz
)
freq = "3min"

df = await stock.bars.get(start, end, freq=freq)
df = stock.restriction.filter(df)
df["atr"] = await stock.bars.get_atr(start, end, freq=freq)
df[["supertrend", "bullish"]] = await stock.bars.get_supertrend(
    start, end, freq, n=10, k=3
)
df[["long_supertrend", "long_bullish"]] = await stock.bars.get_supertrend(
    start, end, freq, n=40, k=5
)
df = stock.restriction.filter(df)


def plot_data(df):
    supertrend, bullish = df["supertrend"], df["bullish"].astype(bool)
    long_supertrend, long_bullish = df["long_supertrend"], df["long_bullish"].astype(
        bool
    )
    layout = go.Layout(
        autosize=False,
        width=1200,
        height=700,
    )
    fig = go.Figure(layout=layout)
    fig = make_subplots(specs=[[{"secondary_y": True}]], figure=fig)
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df["open"],
            high=df["high"],
            low=df["low"],
            close=df["close"],
            name="Candles",
        ),
    )
    lower_supertrend = supertrend[~bullish].reindex(supertrend.index)
    upper_supertrend = supertrend[bullish].reindex(supertrend.index)
    long_lower_supertrend = long_supertrend[~long_bullish].reindex(
        long_supertrend.index
    )
    long_upper_supertrend = long_supertrend[long_bullish].reindex(long_supertrend.index)
    fig.add_trace(
        go.Scatter(
            x=lower_supertrend.index,
            y=lower_supertrend,
            name="Supertrend",
            marker={"color": "#F00"},
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=upper_supertrend.index,
            y=upper_supertrend,
            name="Supertrend",
            marker={"color": "#0F0"},
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=long_lower_supertrend.index,
            y=long_lower_supertrend,
            name="Long Supertrend",
            marker={"color": "#F00"},
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=long_upper_supertrend.index,
            y=long_upper_supertrend,
            name="Long Supertrend",
            marker={"color": "#0F0"},
        ),
    )
    if "positions" in df.columns:
        positions = df["positions"]
        fig.add_trace(
            go.Scatter(x=positions.index, y=positions, name="Positions"),
            secondary_y=True,
        )
    if "strategy" in df.columns:
        layout = go.Layout(
            autosize=False,
            width=1200,
            height=700,
        )
        fig2 = go.Figure(layout=layout)
        fig2 = make_subplots(specs=[[{"secondary_y": True}]], figure=fig2)
        fig2.add_traces(
            [
                go.Scatter(x=df.index, y=df["strategy"], name="strategy"),
                go.Scatter(x=df.index, y=df["benchmark"], name="benchmark"),
                go.Scatter(x=df.index, y=df["trades"], name="trades"),
            ],
            secondary_ys=[False, False, True],
        )
        fig2.update_xaxes(
            rangeslider_visible=True,
            rangebreaks=[
                dict(
                    bounds=["sat", "mon"]
                ),  # hide weekends, eg. hide sat to before mon
                dict(
                    bounds=[16, 9.5], pattern="hour"
                ),  # hide hours outside of 9.30am-4pm
            ],
        )
        fig2.show()

    # fig.add_trace(
    #     go.Scatter(x=df["atr"].index, y=df["atr"], name="ATR"),
    #     secondary_y=True
    # )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
            dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
        ],
    )
    fig.show()

In [83]:
plot_data(df)

In [148]:
from datetime import time


spread = 0.012


async def process_bars(
    start: pd.Timestamp,
    end: pd.Timestamp,
    freq: str = "3min",
    short_n=10,
    long_n=40,
    short_k=3,
    long_k=5,
):
    bars = await stock.bars.get(start, end, freq)
    atr_ser = await stock.bars.get_atr(start, end, freq)
    supertrend_df = await stock.bars.get_supertrend(
        bars.index[0], bars.index[-1], freq, n=short_n, k=short_k
    )
    long_supertrend_df = await stock.bars.get_supertrend(
        bars.index[0], bars.index[-1], freq, n=long_n, k=long_k
    )
    prev_bar, prev_atr, prev_supertrend, prev_long_supertrend = (
        bars.iloc[0],
        atr_ser.iloc[0],
        supertrend_df.iloc[0],
        long_supertrend_df.iloc[0],
    )
    for (index, bar), atr, (_, supertrend), (_, long_supertrend) in zip(
        bars.iloc[1:].iterrows(),
        atr_ser.iloc[1:],
        supertrend_df.iloc[1:].iterrows(),
        long_supertrend_df.iloc[1:].iterrows(),
    ):
        if index.time() > time(15, 30):
            bars.loc[index, "positions"] = 0
            continue

        if supertrend > bar["close"] and long_supertrend > bar["close"]:
            await stock.orders.market_buy(
                size=1, reduce_only=False, ioc=False, post_only=False
            )
        elif supertrend < bar["close"] and long_supertrend < bar["close"]:
            await stock.orders.market_sell(
                size=1, reduce_only=False, ioc=False, post_only=False
            )
    # bars = stock.restriction.filter(bars)
    # supertrend = stock.restriction.filter(supertrend)
    # long_supertrend = stock.restriction.filter(long_supertrend)
    # bars[["supertrend", "bullish"]] = supertrend
    # bars[["long_supertrend", "long_bullish"]] = long_supertrend
    # bars = bars.dropna()
    # bars["trend_change"] = bars["bullish"].diff().fillna(0)
    # bars["long_trend_change"] = bars["long_bullish"].diff().fillna(0)
    # bars.loc[bars["long_trend_change"] != 0, "positions"] = 0
    # long_bullish = bars["long_bullish"].astype(bool)
    # bars.loc[(bars["trend_change"] == 1) & long_bullish, "positions"] = 1
    # bars.loc[(bars["trend_change"] == -1) & ~long_bullish, "positions"] = -1
    # bars.loc[(bars["trend_change"] == 1) & ~long_bullish, "positions"] = 0
    # bars.loc[(bars["trend_change"] == -1) & long_bullish, "positions"] = 0
    # bars.loc[bars.index.time > time.fromisoformat("15:30"),"positions"] = 0
    # bars["positions"] = bars["positions"].ffill().shift(1).fillna(0)
    # bars["trades"] = bars["positions"].diff().abs().fillna(0).cumsum()
    # bars["price_change"] = np.log(bars["close"] / bars["close"].shift(1)).fillna(0)
    # bars["profit"] = bars["price_change"] * bars["positions"]
    # bars["strategy"] = (
    #     np.exp(bars["profit"].cumsum()) - bars["trades"] * spread / bars["close"]
    # )
    # bars["benchmark"] = np.exp(bars["price_change"].cumsum()) - spread / bars["close"]
    return bars

In [149]:
(await process_bars(start_train, end_train)).head(10)

,open,high,low,close,volume,price,supertrend,bullish,long_supertrend,long_bullish,trend_change,long_trend_change,positions,trades,price_change,profit,strategy,benchmark
start,,,,,,,,,,,,,,,,,,
2021-08-27 09:33:00-04:00,147.680,147.6800,147.2150,147.5000,767279,147.454970,147.92223,False,146.75695,True,0,0,0.0,0.0,0.000000,0.0,1.0,0.999919
2021-08-27 09:36:00-04:00,147.480,147.6500,147.2500,147.2550,715831,147.415593,147.92223,False,146.75695,True,0,0,0.0,0.0,-0.001662,-0.0,1.0,0.998257
2021-08-27 09:39:00-04:00,147.260,147.2900,146.9100,146.9150,992629,147.089314,147.92223,False,146.75695,True,0,0,0.0,0.0,-0.002312,-0.0,1.0,0.995952
2021-08-27 09:42:00-04:00,146.910,147.2487,146.8532,147.1400,645050,147.101887,147.92223,False,146.75695,True,0,0,0.0,0.0,0.001530,0.0,1.0,0.997478
2021-08-27 09:45:00-04:00,147.140,147.3100,147.0000,147.0000,948672,147.186674,147.92223,False,146.75695,True,0,0,0.0,0.0,-0.000952,-0.0,1.0,0.996529
2021-08-27 09:48:00-04:00,146.990,147.0450,146.8500,146.8786,671280,146.949853,147.92223,False,146.75695,True,0,0,0.0,0.0,-0.000826,-0.0,1.0,0.995705
2021-08-27 09:51:00-04:00,146.875,147.0400,146.8300,146.8901,584196,146.936821,147.92223,False,146.75695,True,0,0,0.0,0.0,0.000078,0.0,1.0,0.995783
2021-08-27 09:54:00-04:00,146.890,147.0400,146.8301,146.9650,531206,146.936911,147.92223,False,146.75695,True,0,0,0.0,0.0,0.000510,0.0,1.0,0.996291
2021-08-27 09:57:00-04:00,146.968,147.1800,146.9200,147.1800,575574,147.032803,147.92223,False,146.75695,True,0,0,0.0,0.0,0.001462,0.0,1.0,0.997749


In [160]:
import optuna
from threading import Thread
from time import sleep


def objective(trial):
    short_n = trial.suggest_int("short_n", 10, 40)
    long_n = trial.suggest_int("long_n", 40, 200)
    short_k = trial.suggest_float("short_k", 0.5, 3)
    long_k = trial.suggest_float("long_k", short_k, 8)
    df = []

    def func(df):
        result = asyncio.run(
            process_bars(
                start_train,
                end_train,
                freq="20min",
                short_n=short_n,
                long_n=long_n,
                short_k=short_k,
                long_k=long_k,
            ),
        )
        df.append(result)

    thread = Thread(target=func, args=(df,))
    thread.start()
    thread.join()
    #     up_cross_stat = trial.suggest_categorical("up_cross_stat", ["open", "low", "high", "close", "price"])
    #     low_cross_stat = trial.suggest_categorical("low_cross_stat", ["open", "low", "high", "close", "price"])
    df = df[0]
    n_trades = df["trades"][-1]
    if n_trades < 10:
        return 0
    hit_ratio = (df["profit"] > 0).sum() / (df["profit"] != 0).sum()
    # return (df["strategy"][-1] - df["strategy"][0]) / df["strategy"][0]
    return hit_ratio


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40)

[I 2022-09-08 10:10:09,479] A new study created in memory with name: no-name-13c45f09-b3e1-4882-9640-5d6cadae6bc8
[I 2022-09-08 10:10:09,776] Trial 0 finished with value: 0.5366666666666666 and parameters: {'short_n': 30, 'long_n': 178, 'short_k': 1.8323171431821965, 'long_k': 2.180428438286916}. Best is trial 0 with value: 0.5366666666666666.
[I 2022-09-08 10:10:10,013] Trial 1 finished with value: 0.5335365853658537 and parameters: {'short_n': 13, 'long_n': 174, 'short_k': 2.05822250357774, 'long_k': 2.0758036785914578}. Best is trial 0 with value: 0.5366666666666666.
[I 2022-09-08 10:10:10,252] Trial 2 finished with value: 0.5361552028218695 and parameters: {'short_n': 37, 'long_n': 45, 'short_k': 2.5481467974099394, 'long_k': 5.970250981337645}. Best is trial 0 with value: 0.5366666666666666.
[I 2022-09-08 10:10:10,486] Trial 3 finished with value: 0.5460526315789473 and parameters: {'short_n': 40, 'long_n': 192, 'short_k': 2.8435844587309056, 'long_k': 3.259432030608988}. Best is 

In [161]:
study.trials_dataframe().sort_values("value", ascending=False).head(5)

,number,value,datetime_start,datetime_complete,duration,params_long_k,params_long_n,params_short_k,params_short_n,state
21,21,0.560272,2022-09-08 10:10:14.681409,2022-09-08 10:10:14.929109,0 days 00:00:00.247700,1.531527,162,0.828701,33,COMPLETE
34,34,0.556675,2022-09-08 10:10:17.909556,2022-09-08 10:10:18.172322,0 days 00:00:00.262766,1.501918,175,0.692715,38,COMPLETE
25,25,0.554994,2022-09-08 10:10:15.682381,2022-09-08 10:10:15.927508,0 days 00:00:00.245127,1.581370,165,0.683052,33,COMPLETE
20,20,0.553872,2022-09-08 10:10:14.434166,2022-09-08 10:10:14.680873,0 days 00:00:00.246707,1.554542,186,0.872231,33,COMPLETE
26,26,0.551977,2022-09-08 10:10:15.928026,2022-09-08 10:10:16.175102,0 days 00:00:00.247076,1.393078,180,0.693952,28,COMPLETE


In [162]:
from random import random


df = await process_bars(
    start_test, end_test, freq="20min", short_n=30, long_n=150, short_k=2.5, long_k=4
)
n_items = len(df)
s = random() * 0.8
e = s + 0.2 * random()
start, end = df.index[int(n_items * s)], df.index[int(n_items * e)]
# print(df[start:end])
plot_data(df)

In [101]:
df["bullish"]

start
2022-08-17 09:30:00-04:00     True
2022-08-17 09:33:00-04:00     True
2022-08-17 09:36:00-04:00     True
2022-08-17 09:39:00-04:00     True
2022-08-17 09:42:00-04:00     True
                             ...  
2022-08-17 15:48:00-04:00    False
2022-08-17 15:51:00-04:00    False
2022-08-17 15:54:00-04:00    False
2022-08-17 15:57:00-04:00    False
2022-08-17 16:00:00-04:00    False
Freq: 3T, Name: bullish, Length: 131, dtype: bool

DatetimeIndex(['2021-08-27 14:33:00-04:00', '2021-08-27 14:36:00-04:00',
               '2021-08-27 14:39:00-04:00', '2021-08-27 14:42:00-04:00',
               '2021-08-27 14:45:00-04:00', '2021-08-27 14:48:00-04:00',
               '2021-08-27 14:51:00-04:00', '2021-08-27 14:54:00-04:00',
               '2021-08-27 14:57:00-04:00', '2021-08-27 15:00:00-04:00',
               ...
               '2022-06-14 13:27:00-04:00', '2022-06-14 13:30:00-04:00',
               '2022-06-14 13:33:00-04:00', '2022-06-14 13:36:00-04:00',
               '2022-06-14 13:39:00-04:00', '2022-06-14 13:42:00-04:00',
               '2022-06-14 13:45:00-04:00', '2022-06-14 13:48:00-04:00',
               '2022-06-14 13:51:00-04:00', '2022-06-14 13:54:00-04:00'],
              dtype='datetime64[ns, US/Eastern]', name='start', length=26187, freq=None)